# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [141]:
# Libraries
import numpy as np
import pandas as pd

import re

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats
from sklearn.linear_model import LinearRegression

# Challenge 1 - Index Functions

In this challenge, we will practice the advanced functions introduced in the lesson. However, before we start, we will load and evaluate our dataset. 

Load the dataset from Ironhack's database:
* db: `admissions predict`
* table: `admissions predict`

In [142]:
# your code here

path = "C:/Users/haenni/Documents/GitHub/lab-advanced-pandas/your-code/data/Admission_Predict.csv"
ds = pd.read_csv(path)
ds.head()

ds.columns.values

array(['Serial No.', 'GRE Score', 'TOEFL Score', 'University Rating',
       'SOP', 'LOR ', 'CGPA', 'Research', 'Chance of Admit '],
      dtype=object)

Remove trailing spaces at the end of the column names if there are any.


In [143]:
# your code here

colNames = {c: c.rstrip() for c in ds.columns}

ds= ds.rename(columns = colNames)
ds.columns.values

array(['Serial No.', 'GRE Score', 'TOEFL Score', 'University Rating',
       'SOP', 'LOR', 'CGPA', 'Research', 'Chance of Admit'], dtype=object)

Let's evaluate the dataset by looking at the `head` function.

In [144]:
# your code here
ds.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90


Before beginning to work with this dataset and evaluating graduate admissions data, we will verify that there is no missing data in the dataset. Do this in the cell below.

In [145]:
# your code here
ds.isnull().sum()

Serial No.           0
GRE Score            0
TOEFL Score          0
University Rating    0
SOP                  0
LOR                  0
CGPA                 0
Research             0
Chance of Admit      0
dtype: int64

Interestingly, there is a column that uniquely identifies the applicants. This column is the serial number column. Instead of having our own index, we should make this column our index. Do this in the cell below. Keep the column in the dataframe in addition to making it an index.

In [146]:
# your code here
ds= ds.set_index('Serial No.', drop = False)
ds.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
Serial No.,,,,,,,,,
1,1,337,118,4,4.5,4.5,9.65,1,0.92
2,2,316,104,3,3.0,3.5,8.00,1,0.72
3,3,322,110,3,3.5,2.5,8.67,1,0.80
4,4,314,103,2,2.0,3.0,8.21,0,0.65
5,5,330,115,5,4.5,3.0,9.34,1,0.90


Turns out that `GRE Score` and `CGPA` also uniquely identify the data. Show this in the cell below.

In [147]:
# your code here

print("rows in ds:" + str(ds.shape[0]))

np.unique(ds[['GRE Score', 'CGPA']], axis=0).size /2


rows in ds:385


385.0

Replace the index with an index comprised of two columns - `GRE Score` and `CGPA`. Remove the columns from the dataset as well. Make sure to use `inplace=True`.

In [148]:
ds.set_index(['GRE Score', 'CGPA'], inplace=True)
ds.head()

,,Serial No.,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit
GRE Score,CGPA,,,,,,,
337,9.65,1,118,4,4.5,4.5,1,0.92
316,8.00,2,104,3,3.0,3.5,1,0.72
322,8.67,3,110,3,3.5,2.5,1,0.80
314,8.21,4,103,2,2.0,3.0,0,0.65
330,9.34,5,115,5,4.5,3.0,1,0.90


Now change the index back to a sequence starting at zero using the `reset_index` function. Make sure to use `inplace=True`.

In [149]:
# your code here
ds.reset_index(inplace=True)
ds.head()

,GRE Score,CGPA,Serial No.,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit
0,337,9.65,1,118,4,4.5,4.5,1,0.92
1,316,8.00,2,104,3,3.0,3.5,1,0.72
2,322,8.67,3,110,3,3.5,2.5,1,0.80
3,314,8.21,4,103,2,2.0,3.0,0,0.65
4,330,9.34,5,115,5,4.5,3.0,1,0.90


# Challenge 2 - Advanced Functions

In this part of the lab, we would like to test complex conditions on the entire dataset at once. Let's start by finding the number of rows where the CGPA is higher than 9 and the student has conducted research.

In [150]:
# your code here

ds.loc[(ds['CGPA']> 9.0) & (ds['Research']==1)]

## what do you men by the number of rows? the length?
ds_sub.shape[0]

101

Now return all the rows where the CGPA is greater than 9 and the SOP score is less than 3.5. Find the mean chance of admit for these applicants.

In [151]:
# your code here
ds.loc[(ds['CGPA']> 9.0) & (ds['SOP']< 3.5)]

,GRE Score,CGPA,Serial No.,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit
28,338,9.40,29,118,4,3.0,4.5,1,0.91
62,327,9.02,63,114,3,3.0,3.0,0,0.61
140,326,9.11,141,114,3,3.0,3.0,1,0.83
217,324,9.01,218,111,4,3.0,3.0,1,0.82
381,325,9.11,382,107,3,3.0,3.5,1,0.84


We would like to create a deciding factor column for each student. We standardize several columns and then pick the most important factor from a lookup table. If the standardized value is above 0.8, the student will be accepted.

We will start by creating a standardized column for `CGPA`, `GRE Score` and `LOR`. We will name these columns `CGPA_std`, `GRE_std`, and `LOR_std` respecively.

Recall that standardizing a column is done by subtracting the mean of the column from all observations in the column and then dividing each observation in the column by the column's standard deviation.

In the cell below, write a standardization function (a function that takes a column as input and returns a standardized column as output). Make sure to use the `numpy` versions of mean and standard deviation.

In [152]:
def standardize(col):
    """
    This function takes a column from a dataframe 
    and returns a standardized column by subtracting the column's mean
    and dividing by the column's standard deviation.
    """
    
    # your code here
    mean = col.mean()
    std = col.std()
    
    col_std= col.apply(lambda x: (x - mean)/std)
    return col_std

Now create the standardized columns `CGPA_std`, `GRE_std`, and `LOR_std` and add them to the `admissions` dataframe.

In [155]:
# your code here

ds['CGPA_std'] = standardize(ds['CGPA'])
ds['GRE_std'] = standardize(ds['GRE Score'])
ds['LOR_std'] = standardize(ds['LOR'])
ds.head()

,GRE Score,CGPA,Serial No.,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit,CGPA_std,GRE_std,LOR_std
0,337,9.65,1,118,4,4.5,4.5,1,0.92,1.747900,1.753381,1.191647
1,316,8.00,2,104,3,3.0,3.5,1,0.72,-0.991211,-0.063367,0.076740
2,322,8.67,3,110,3,3.5,2.5,1,0.80,0.121034,0.455704,-1.038166
3,314,8.21,4,103,2,2.0,3.0,0,0.65,-0.642597,-0.236391,-0.480713
4,330,9.34,5,115,5,4.5,3.0,1,0.90,1.233279,1.147798,-0.480713


We will generate the decision choice at random using the code below. Please run the cell.

In [156]:
# Libraries
from random import choices

In [163]:
std_columns = ['CGPA_std', 'GRE_std', 'LOR_std']

ds['decision_choice'] = choices(std_columns, k=ds.shape[0])

Now create the deciding column using the `lookup` function. The lookup column is `decision_choice` found above. Call the column resulting from the lookup function `deciding_column` and add it to the `admissions` dataframe.

In [167]:
# your code here

ds['deciding_column'] = ds.lookup(ds.index, ds['decision_choice'])
ds.head()

,GRE Score,CGPA,Serial No.,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit,CGPA_std,GRE_std,LOR_std,decision_choice,deciding_column
0,337,9.65,1,118,4,4.5,4.5,1,0.92,1.747900,1.753381,1.191647,GRE_std,1.753381
1,316,8.00,2,104,3,3.0,3.5,1,0.72,-0.991211,-0.063367,0.076740,GRE_std,-0.063367
2,322,8.67,3,110,3,3.5,2.5,1,0.80,0.121034,0.455704,-1.038166,LOR_std,-1.038166
3,314,8.21,4,103,2,2.0,3.0,0,0.65,-0.642597,-0.236391,-0.480713,CGPA_std,-0.642597
4,330,9.34,5,115,5,4.5,3.0,1,0.90,1.233279,1.147798,-0.480713,GRE_std,1.147798


Create a column called `decision` in the `admissions` dataframe. Assign 1 to this column if the value of `deciding_column` is greater than 0.8 and 0 otherwise.

In [172]:
# your code here
ds['decision'] = ds['deciding_column'].apply(lambda x: 1 if x >0.8 else 0) 

ds.head()

,GRE Score,CGPA,Serial No.,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit,CGPA_std,GRE_std,LOR_std,decision_choice,deciding_column,decision
0,337,9.65,1,118,4,4.5,4.5,1,0.92,1.747900,1.753381,1.191647,GRE_std,1.753381,1
1,316,8.00,2,104,3,3.0,3.5,1,0.72,-0.991211,-0.063367,0.076740,GRE_std,-0.063367,0
2,322,8.67,3,110,3,3.5,2.5,1,0.80,0.121034,0.455704,-1.038166,LOR_std,-1.038166,0
3,314,8.21,4,103,2,2.0,3.0,0,0.65,-0.642597,-0.236391,-0.480713,CGPA_std,-0.642597,0
4,330,9.34,5,115,5,4.5,3.0,1,0.90,1.233279,1.147798,-0.480713,GRE_std,1.147798,1


How many applicants will be accepted to the program using the decision column? Compute the result below.

In [174]:
# your code here
ds.decision.sum()

89

# Challenge 3 - Method Chaining

To increase our coding efficiency, let's make a number of changes to our dataframe in one line of code.

In the cell below, remove all non character symbols from the column names, replace all spaces in column names to underscores, and change all upper case characters to lower case. Assign these new column values to `admissions.columns`.

In [189]:
# your code here

colNames = {c: c.replace("[^A-Za-z0-9]", "").replace(' ','_').lower() for c in ds.columns}

ds= ds.rename(columns = colNames)
ds.columns.values

array(['gre_score', 'cgpa', 'serial_no.', 'toefl_score',
       'university_rating', 'sop', 'lor', 'research', 'chance_of_admit',
       'cgpa_std', 'gre_std', 'lor_std', 'decision_choice',
       'deciding_column', 'decision', 'gre_scorete', 'gre_score2'],
      dtype=object)

In the cell below, give all student with a university rating of 4 or higher a 10 point boost on their GRE score and split the column into 4 bins using the `cut` function. Assign this new score to the variable `adjusted_gre`.

In [191]:
# your code here

ds['adjusted_gre'] = ds['gre_score']+ ds['university_rating'].apply(lambda x: 10 if x >4 else 0) 
mpg_labels=[1,2,3,4]
ds['adjusted_gre_bin'] = pd.cut(ds['adjusted_gre'], 4, labels = mpg_labels)

ds.head() 

,gre_score,cgpa,serial_no.,toefl_score,university_rating,sop,lor,research,chance_of_admit,cgpa_std,gre_std,lor_std,decision_choice,deciding_column,decision,gre_scorete,gre_score2,adjusted_gre,adjusted_gre_bin
0,337,9.65,1,118,4,4.5,4.5,1,0.92,1.747900,1.753381,1.191647,GRE_std,1.753381,1,674,337,337,4
1,316,8.00,2,104,3,3.0,3.5,1,0.72,-0.991211,-0.063367,0.076740,GRE_std,-0.063367,0,632,316,316,2
2,322,8.67,3,110,3,3.5,2.5,1,0.80,0.121034,0.455704,-1.038166,LOR_std,-1.038166,0,644,322,322,3
3,314,8.21,4,103,2,2.0,3.0,0,0.65,-0.642597,-0.236391,-0.480713,CGPA_std,-0.642597,0,628,314,314,2
4,330,9.34,5,115,5,4.5,3.0,1,0.90,1.233279,1.147798,-0.480713,GRE_std,1.147798,1,660,340,340,4
